# Slamseq Ibet re-analysis from Muhar Paper

In [ ]:
from __future__ import print_function
import os.path
import pandas as pd
import sys
sys.path.insert(0, '../../')
import seaborn as sns
import numpy as np

from JKBio.Helper import *
from JKBio.helper import pyDESeq2

from bokeh.plotting import *
from bokeh.models import HoverTool
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.manifold import MDS, TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
#from umap import UMAP

output_notebook()
%load_ext autoreload
%matplotlib inline
%autoreload 2
%load_ext rpy2.ipython

downloading their data

## downloading the data

In [ ]:
version="v4"
project="iBet_slamseq"
location= '../data/'+project+"/"

In [ ]:
gsutil -m cp -r gsutil ls gs://amlproject/RNA/slamseq/MYCpaper/ ../../data/$project/

In [ ]:
location= '../../data/'+project+'/count/'

In [ ]:
files = ! ls $location*.tsv
files

## we first need to recreate out RNA matrix from slamdunk's tcount files

In [ ]:
data={}
for file in files:
    data[file.split('.')[-3].split('/')[-1]]= pd.read_csv(file, sep='\t', comment='#',header=0)

In [ ]:
prev = -2
for k,val in data.items():
    if len(set(val.Name)) != prev and prev!=-2: 
        raise ValueError('we do not have the same number of genes in each file')
    prev = len(set(val.Name))

![](images/conversionrates.png)

In [ ]:
data.keys()

In [ ]:
val

In [ ]:
flag_var= 100

readcounts = {i:[0]*len(data) for i in val.Name.unique()}
tccounts = {i:[0]*len(data) for i in val.Name.unique()}

In [ ]:
for n, (_,val) in enumerate(data.items()):
    val = val.sort_values(by="Name")
    j  = 0
    #print('              ',end='\r')
    readcount = [val.iloc[0].ReadCount]
    tccount = [val.iloc[0].TcReadCount]
    prevname = val.iloc[0].Name
    for _, v in val.iloc[1:].iterrows():
        if v.Name==4609:
            print(v.ReadCount,v.TcReadCount)
            print(readcount,tccount)
        if v.Name == prevname:
            readcount.append(v.ReadCount)
            tccount.append(v.TcReadCount)
        else:
            readcounts[prevname][n] = np.sum(readcount)
            tccounts[prevname][n] = np.sum(tccount)
            #if np.var(readcount) > flag_var:
            #    print("pb with "+str(v.Name))
            prevname=v.Name
            j+=1
            #print(j,end='\r')
            readcount = [v.ReadCount]
            tccount = [v.TcReadCount]
            
files = [*data]

In [ ]:
val.Name

In [ ]:
readcounts = pd.DataFrame(data=readcounts,columns=val.Name.unique(),index=data.keys()).T
tccounts = pd.DataFrame(data=tccounts,columns=val.Name.unique(),index=data.keys()).T

In [ ]:
names, _ = convertGenes(readcounts.index.tolist(), from_idtype="entrez_id", to_idtype="symbol")
readcounts.index = names

In [ ]:
names, _ = convertGenes(tccounts.index.tolist(), from_idtype="entrez_id", to_idtype="symbol")
tccounts.index = names

In [ ]:
tccounts.values.mean()

In [ ]:
minvar_toremove = 2
mincount_toremove = 10

In [ ]:
nottodrop = np.argwhere(tccounts.values.var(0)>=lowvar_toremove).ravel()
#tccounts = tccounts.iloc[nottodrop]
#readcounts = readcounts.iloc[nottodrop]

In [ ]:
nottodrop=np.argwhere(readcounts.values.max(1)>=mincount_toremove).ravel()
tccounts = tccounts.iloc[nottodrop]
readcounts = readcounts.iloc[nottodrop]

In [ ]:
len(readcounts)

## SLAM-seq
All SLAM-seq assays were performed at 60-70% confluency for adherent cells or
60% of the maximum cell density counted on a hemocytometer for suspension cells. 5-7h
prior to each assay, growth medium was aspirated and replaced. Unless stated otherwise,
cells were pre-treated with indicated small molecule inhibitors or 100µM IAA for 30 min
to pre-establish full target inhibition or degradation. Newly synthesized RNA was labeled
for indicated time spans (45 min or 60 min) at a final concentration of 100µM 4-
thiouridine (4sU, Carbosynth). Adherent cells were harvested by direct snap-freezing of
plates on dry ice. Suspension cells were spun down and immediately snap-frozen. RNA
extraction was performed using the RNeasy Plus Mini Kit (Qiagen). 

Total RNA was subjected to alkylation by iodoacetamide (Sigma, 10mM) for 15 min and RNA was repurified by ethanol precipitation. 500ng alkylated RNA were used as input for generating
3’-end mRNA sequencing libraries using a commercially available kit (QuantSeq 3′
mRNA-Seq Library Prep Kit FWD for Illumina and PCR Add-on Kit for Illumina,
Lexogen). Deep sequencing was performed using HiSeq1500 and HiSeq2500 platforms
(Illumina).

0.06724463
0.04916348
0.03728357
0.05075007
0.03331248
0.0346662
0.03265504 
0.04416262
0.05574821
0.05845953
0.05531433
0.06175393
0.03181749 
0.02940226 
0.0306263
0.04004278

In [ ]:
tccounts

In [ ]:
pd.DataFrame(tccounts.values[:,:-1].astype(float)/readcounts.mean().values*100, columns=tccounts.columns[:-1], index=tccounts['genes']).to_csv('../results/'+project+'/'version+'_'+minvar_toremove+'_'+mincount_toremove+'_ratio.csv')

In [ ]:
(tccounts.values[:,:-1].astype(float)/(1+readcounts.values[:,:-1].astype(float))).mean(0)

In [ ]:
ratio.mean()

In [ ]:
tccounts.mean()

In [ ]:
readcounts.to_csv('../results/'+project+'/'version+'_'+minvar_toremove+'_'+mincount_toremove+'_readcounts.csv',index=False)

In [ ]:
tccounts.to_csv('../results/'+project+'/'version+'_'+minvar_toremove+'_'+mincount_toremove+'_tccounts.csv',index=False)

### make more dataframes

In [ ]:
mtccounts = pd.DataFrame()
mreadcounts = pd.DataFrame()
for i in range(4):
    mtccounts[tccounts.columns[i*3].split("_rep")[0]] = tccounts[tccounts.columns[(i*3):(i*3)+3]].mean(1)
    mreadcounts[tccounts.columns[i*3].split("_rep")[0]] = readcounts[readcounts.columns[(i*3):(i*3)+3]].mean(1)

In [ ]:
mtccounts.to_csv("../data/"+project+'/'version+'_'+minvar_toremove+'_'+mincount_toremove+"_mean_tccounts.csv")
mreadcounts.to_csv("../data/"+project+'/'version+'_'+minvar_toremove+'_'+mincount_toremove+"_mean_readcounts.csv")

In [ ]:
mtccounts = pd.DataFrame()
mreadcounts = pd.DataFrame()
for i in range(4):
    mtccounts[tccounts.columns[i*3].split("_rep")[0]] = tccounts[tccounts.columns[(i*3):(i*3)+3]].median(1)
    mreadcounts[tccounts.columns[i*3].split("_rep")[0]] = readcounts[readcounts.columns[(i*3):(i*3)+3]].median(1)

In [ ]:
mtccounts.to_csv("../data/"+project+'/'version+'_'+minvar_toremove+'_'+mincount_toremove+"_median_tccounts.csv")
mreadcounts.to_csv("../data/"+project+'/'version+'_'+minvar_toremove+'_'+mincount_toremove+"_median_readcounts.csv")

In [ ]:
(tccounts/readcounts).fillna(0).to_csv("../data/"+project+'/'version+'_'+minvar_toremove+'_'+mincount_toremove+"_tccounts_per_readcounts.csv")

# lets look at the similarity accross replicates

we have some outliers, else it seems to make some sense and on average, to be extremelly similar!

In [ ]:
counts.loc['MYC']

In [ ]:
tccounts.columns = [i.replace('-','.') for i in tccounts.columns]
readcounts.columns = [i.replace('-','.') for i in readcounts.columns]

In [ ]:
%matplotlib inline
sort = [0,8,10,4,6,14,12,1,11,9,5,7,13,3,15,2]
sns.heatmap(tccounts[tccounts.columns[sort]].corr(), 
            xticklabels=tccounts.columns[sort].values,
            yticklabels=tccounts.columns[sort].values)
plt.savefig('../results/'+project+'/plots/'+version+'_'+minvar_toremove+'_'+mincount_toremove+'_similarity_replicates_sorted_readcounts.csv')

In [ ]:
%matplotlib inline
sns.heatmap(readcounts.corr(), 
            xticklabels=readcounts.columns,
            yticklabels=readcounts.columns)
plt.savefig('../results/'+project+'/plots/'+version+'_similarity_replicates.csv')

In [ ]:
%matplotlib inline
sort = [0,8,10,4,12,6,14,1,9,5,13,7,11,15,3,2]
sns.heatmap(readcounts[readcounts.columns[sort]].corr(), 
            xticklabels=readcounts.columns.values[sort],
            yticklabels=readcounts.columns.values[sort])
plt.savefig('../results/'+project+'/plots/'+version+'_'+minvar_toremove+'_'+mincount_toremove+'_similarity_replicates_sorted_tccounts.csv')

read the paper for MV411 QC steps.

### A quick detour to look for clustering

## Getting the Core TF information

In [ ]:
ctf=pd.read_csv('../data/CTF.csv',header=None)[0].values.tolist()
ctf

In [ ]:
set(ctf) & set(tccounts.index)

In [ ]:
ctfpos = [val for val in tccounts.index if val in ctf]
notctfpos = [val for val in tccounts.index if val not in ctf]

In [ ]:
ctfpos.extend(['EGR1','SERTAD1'])

We find a CTF not in the dataset

# Let's look at how they change accross time

how core transcription factors change accross time when the cell is with JQ1 conditions

we are looking at the difference between production of each CTF compared to the average RNA production with JQ1 and without JQ1

## DESEQ ANALYSIS

In [ ]:
readcounts["genes"] = readcounts.index
readcounts = readcounts.reset_index(drop=True)
tccounts["genes"] = tccounts.index
tccounts = tccounts.reset_index(drop=True)

In [ ]:
tccounts.iloc[140]

In [ ]:
tccounts.columns

In [ ]:
design = pd.DataFrame(index=[i.replace('-','.') for i in tccounts.columns.values[:-1]], 
                      columns=['VHL','DMSO'], 
                      data=np.array([[1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0],
                                     [0,0,0,0,1,1,1,0,0,0,0,0,0,1,0,0]],dtype=bool).T)

In [ ]:
design_reg = pd.DataFrame(index=[i.replace('-','.') for i in tccounts.columns.values[:-1]], 
                      columns=['DMSO','VHL'], 
                      data=np.array([[0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0],
                                     [0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1]],dtype=bool).T)

In [ ]:
designoto = pd.DataFrame(index=[i.replace('-','.') for i in tccounts.columns.values[:-1]], 
                      columns=['DMSO','VHL'], 
                      data=np.array([[1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0],
                                     [0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0]],dtype=bool).T)

### Differential gene expression analysis, PCA and GO-term enrichment

For gene-level analysis, raw reads mapped to different UTR annotations of the same gene were summed up by Entrez Gene ID. Pilot studies of K562 cells with kinase inhibitors were performed as single experiments. 

Analysis of differential gene expression was restricted to genes with ≥ 10 reads in at least one condition for 50bp sequencing runs (flavopiridol and DMSO) or ≥ 20 reads in at least one condition for 100bp sequencing runs (mk2206, trametinib, nilotinib, trametinib + mk2206 and DMSO). For estimating differential expression, a pseudo-count of 1 raw read was added to all genes.

#### All other SLAM-seq experiments were performed in triplicates and analyzed as follows. 
Differential gene expression calling was performed on raw read counts with ≥ 2 T>C conversions using DESeq2 (version 1.14.1) with default settings, and with size factors estimated on corresponding total mRNA reads for global normalization.

Downstream analysis was restricted to genes passing all internal filters for FDR estimation by DESeq2. Principal component analysis was performed after variance
stabilizing transformation on the 500 most variable genes across all conditions of a given
experiment. GO-term enrichment analysis was performed on genes significantly and strongly downregulated (FDR ≤ 0.1, log2FC ≤ -1) in SLAM-seq upon IAA-treatment in K562MYC-AID + Tir1 by the PANTHER Overrepresentation Test (Fisher's Exact with FDR multiple test correction, release 20171205, http://pantherdb.org) on GO Ontology database Released 2017-12-27.

In [ ]:
design

In [ ]:
tccounts.columns = [i.replace('-','.') for i in tccounts.columns]

In [ ]:
tccounts[tccounts.genes=='MYC']

In [ ]:
deseq = pyDESeq2.pyDESeq2(count_matrix=tccounts[tccounts.columns[np.array([1,0,0,0,1,1,1,0,1,1,1,0,0,1,0,0,1],np.bool)]],
                         design_matrix=design[np.array([1,0,0,0,1,1,1,0,1,1,1,0,0,1,0,0],np.bool)],
                         design_formula="~DMSO - VHL",
                         gene_column="genes")

In [ ]:
deseq = pyDESeq2.pyDESeq2(count_matrix=tccounts[tccounts.columns[np.array([0,0,0,0,1,1,1,1,0,0,0,0,1,1,1,1,1],np.bool)]],
                         design_matrix=design_reg[np.array([0,0,0,0,1,1,1,1,0,0,0,0,1,1,1,1],np.bool)],
                         design_formula="~DMSO - VHL",
                         gene_column="genes")

In [ ]:
deseq = pyDESeq2.pyDESeq2(count_matrix=tccounts[tccounts.columns[np.array([1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,1],np.bool)]],
                         design_matrix=designoto[np.array([1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0],np.bool)],
                         design_formula="~DMSO - VHL",
                         gene_column="genes")

In [ ]:
np.exp(np.mean(np.log(readcounts[readcounts.columns[np.array(
    [1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0],np.bool)]].values+1), 1))

In [ ]:
deseq.run_estimate_size_factors(geoMeans = np.exp(np.mean(np.log(
    readcounts[readcounts.columns[
        np.array([1,0,0,0,1,1,1,0,1,1,1,0,0,1,0,0,0],np.bool)]].values+1), 1)))

In [ ]:
deseq.run_estimate_size_factors(geoMeans = np.exp(np.mean(np.log(
    readcounts[readcounts.columns[
        np.array([0,0,0,0,1,1,1,1,0,0,0,0,1,1,1,1,0],np.bool)]].values+1), 1)))

In [ ]:
deseq.run_estimate_size_factors(geoMeans = np.exp(np.mean(np.log(
    readcounts[readcounts.columns[
        np.array([1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0],np.bool)]].values+1), 1)))

In [ ]:
deseq.run_deseq()

In [ ]:
deseq.get_deseq_result()

In [ ]:
JQ1res = deseq.deseq_result

In [ ]:
JQ1res.pvalue = np.nan_to_num(np.array(JQ1res.pvalue), 1)
JQ1res.log2FoldChange = np.nan_to_num(np.array(JQ1res.log2FoldChange), 0)

In [ ]:
JQ1res[JQ1res.genes=='MYC']

In [ ]:
JQ1res.to_csv("../results/"+project+"/"+version+'_'+minvar_toremove+'_'+mincount_toremove+'_jq1_deseq.csv")

In [ ]:
JQ1res.log2FoldChange = -JQ1res.log2FoldChange

In [ ]:
JQ1res["gene_id"] = JQ1res.genes

In [ ]:
JQ1res[JQ1res.gene_id=='HNRNPA1']

In [ ]:
volcano(JQ1res,tohighlight=ctf, searchbox=True, title='DESeq results of MV411 under JQ1 in volcano plot',folder='../data/'+project+'/plots/'+version+'_'+minvar_toremove+'_'+mincount_toremove)

we can conclude that we get similar results to the slamseq myc paper although it seems that our values are a bit skewed toward higher expression than what is on the slamseq paper. It mightt be explained by the pseudo count of 1 that I did not set. Because I think it would highly bias the DESeq algorithm.